**選取資料(在各airbox 資料依各縣市分配)**

In [ ]:
with open(r"E:\D4SG\Data\PM2.5 airbox\PM2.5國網與中研院移除飄移.csv", encoding = "utf8") as pm25:
    line = pm25.readline()
    want_address = {}    
    remove_address = []  
    google_address = {}  
      
    while line:
        line = line.strip().split(",")
        
        try: 
            # the scope of latitude and longitude
            if (21.890036 < float(line[7]) < 22.902293) & (120.315043 < float(line[8]) <120.919145):
                station_id = line[2]
                 
                if station_id in remove_address:
                    line = pm25.readline()
                    continue
                
                elif (station_id not in want_address) and (station_id not in google_address):
                    try:
                        address = geocoder.arcgis(line[7:9], method = "reverse")
                        
                        if address.error == False:
                            address = address.geojson['features'][0].get('properties').get("raw").get("address").get('LongLabel')
                            
                        else:
                            while address.error == True:
                                print("sleep.arcgis")
                                time.sleep(5)
                                address = geocoder.arcgis(line[7:9], method = "reverse")
                            
                            address = address.geojson['features'][0].get('properties').get("raw").get("address").get('LongLabel')    
                        
                    except IndexError:
                        address = geocoder.google(line[7:9], method = "reverse", language="zh-TW", timeout=10)
                        
                        if address.error == False:
                            address = address.current_result
                            address = str(address).strip("[").strip("]")
                        
                        else:
                            while address.error == True:
                                print("sleep.google")
                                time.sleep(5)
                                address = geocoder.google(line[7:9], method = "reverse", language="zh-TW", timeout=10)                          
                            
                            address = address.current_result  
                            address = str(address).strip("[").strip("]")
                        
                    if ("屏東" in address):   
                        want_address[station_id] = address

                    elif ("屏東" in address):
                        google_address[station_id] = address
                                                
                    else:   
                        remove_address.append(station_id)
                        line = pm25.readline()
                        continue
                    
                    line.append(address)
                    outputFile = open(r'E:\D4SG\Data\PM2.5 airbox\PM2.5國網與中研院屏東縣.csv', 'a', newline='', encoding = "utf8")
                    outputWriter = csv.writer(outputFile)
                    outputWriter.writerow(line)
                    outputFile.close()    
                    
                elif station_id in want_address:
                    line.append(want_address.get(station_id))
                    outputFile = open(r'E:\D4SG\Data\PM2.5 airbox\PM2.5國網與中研院屏東縣.csv', 'a', newline='', encoding = "utf8")
                    outputWriter = csv.writer(outputFile)
                    outputWriter.writerow(line)
                    outputFile.close()
                
                elif station_id in google_address:
                    line.append(google_address.get(station_id))
                    outputFile = open(r'E:\D4SG\Data\PM2.5 airbox\PM2.5國網與中研院屏東縣.csv', 'a', newline='', encoding = "utf8")
                    outputWriter = csv.writer(outputFile)
                    outputWriter.writerow(line)
                    outputFile.close()
        
        except ValueError:
            line.append("地址")
            outputFile = open(r'E:\D4SG\Data\PM2.5 airbox\PM2.5國網與中研院屏東縣.csv', 'a', newline='', encoding = "utf8")
            outputWriter = csv.writer(outputFile)
            outputWriter.writerow(line)
            outputFile.close()
        
        line = pm25.readline()


**資料轉小時(小時資料的提供)**

In [ ]:
file = ['PM2.5國網與中研院南投縣.csv', 'PM2.5國網與中研院台中市.csv', 'PM2.5國網與中研院台南市.csv', 'PM2.5國網與中研院嘉義縣(市).csv', 
        'PM2.5國網與中研院基隆市.csv', 'PM2.5國網與中研院大臺北(台北+新北).csv', 'PM2.5國網與中研院屏東縣.csv', 'PM2.5國網與中研院彰化縣(市).csv', 
         'PM2.5國網與中研院新竹縣(市).csv', 'PM2.5國網與中研院桃園市.csv', 'PM2.5國網與中研院苗栗縣.csv', 'PM2.5國網與中研院雲林縣.csv', 
        'PM2.5國網與中研院高雄市.csv']

import pandas as pd
for i in file:
    result_table = []
    with open(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}".format(i), encoding = "utf8") as openfile:
        data_temp = pd.read_csv(openfile, skipinitialspace = True)
        
        device_id = data_temp.device_id.unique().tolist()
        for id in device_id:
            revised = data_temp[data_temp['device_id'] == id]
            address = revised["地址"].unique().tolist()[0]
            latitude = revised["latitude"].unique().tolist()[0]
            longitude = revised["longitude"].unique().tolist()[0]

            revised = revised[["device_id", "PM2.5(ug/m3)", "PM10", "temperature", "humidity", "new_time"]]
            revised["time"] = pd.to_datetime(revised["new_time"])
            
            revised = revised.set_index("time").resample('H').mean().interpolate(method = "linear")
            revised["device_id"] = id
            revised["latitude"] = latitude
            revised["longitude"] = longitude
            revised["地址"] = address
            
            revised = revised[["device_id", "PM2.5(ug/m3)", "PM10", "temperature", "humidity", "latitude", "longitude", "地址"]]
            result_table.append(revised)
        
        result = pd.concat(result_table)
        
        name = i.strip("PM2.5國網與中研院").strip(".csv")
        result.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}(每小時).csv".format(name))

**特定季節(15min 資料提供)**

In [ ]:
file_1 = ['PM2.5國網與中研院台中市.csv', 'PM2.5國網與中研院基隆市.csv', 'PM2.5國網與中研院大臺北(台北+新北).csv',
          'PM2.5國網與中研院高雄市.csv']

file_2 = ['PM2.5國網與中研院南投縣.csv', 'PM2.5國網與中研院台中市.csv', 'PM2.5國網與中研院彰化縣(市).csv', 
          'PM2.5國網與中研院苗栗縣.csv']

import pandas as pd
for file in file_1:
    with open(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}".format(file), encoding = "utf8") as pm25:
        pm25_data = pd.read_csv(pm25, low_memory = False)
        
        file = file.strip(".csv")
        table_s1 = []
        table_s2 = []
        table_s3 = []
        table_w1 = []
        table_w2 = []
        table_w3 = []
        want_ids = pm25_data.device_id.unique().copy().tolist()
        
        for want_id in want_ids:
            revised_pd = pm25_data[pm25_data['device_id'] == want_id]
            revised_pd = revised_pd[["new_time", "device_id", "PM2.5(ug/m3)"]]
            revised_pd["new_time"] = pd.to_datetime(revised_pd["new_time"])
            revised_pd = revised_pd.set_index("new_time").resample('15min').mean().interpolate(method = "linear")
            revised_pd[want_id] = revised_pd["PM2.5(ug/m3)"]

            # 夏季分交通尖峰時段
            table_summer = revised_pd.loc['2017-06-15':"2017-09-04"]
            date_delete = ["2017-07-26", "2017-07-27", "2017-07-28", "2017-07-29", "2017-07-30", "2017-07-31", 
                           "2017-08-01", "2017-08-02", "2017-08-18", "2017-08-19", "2017-08-20", "2017-08-21", 
                           "2017-08-22", "2017-08-23", "2017-08-24"]
            table_summer = table_summer.loc[~table_summer.index.isin(date_delete)]
            stime_1 = table_summer.between_time('07:00', '08:59')
            stime_2 = table_summer.between_time('09:00', '15:59')
            stime_3 = table_summer.between_time('16:00', '18:59')

            stime_1 = stime_1[[want_id]]
            table_s1.append(stime_1)     

            stime_2 = stime_2[[want_id]]
            table_s2.append(stime_2)
        
            stime_3 = stime_3[[want_id]]
            table_s3.append(stime_3)
        
            #冬季分交通尖峰時段
            table_winter = revised_pd.loc['2017-01-01':"2017-01-31"]
            wtime_1 = table_winter.between_time('07:00', '08:59')
            wtime_2 = table_winter.between_time('09:00', '15:59')
            wtime_3 = table_winter.between_time('16:00', '18:59')

            wtime_1 = wtime_1[[want_id]]
            table_w1.append(wtime_1)
      
            wtime_2 = wtime_2[[want_id]]
            table_w2.append(wtime_2)
        
            wtime_3 = wtime_3[[want_id]]
            table_w3.append(wtime_3)
                
        result_s1 = pd.concat(table_s1, axis=1)
        result_s1 = result_s1.dropna(axis = 1)
        result_s2 = pd.concat(table_s2, axis=1)
        result_s2 = result_s2.dropna(axis = 1)
        result_s3 = pd.concat(table_s3, axis=1)
        result_s3 = result_s3.dropna(axis = 1)
        
        result_w1 = pd.concat(table_w1, axis=1)
        result_w1 = result_w1.dropna(axis = 1)
        result_w2 = pd.concat(table_w2, axis=1)
        result_w2 = result_w2.dropna(axis = 1)
        result_w3 = pd.concat(table_w3, axis=1)
        result_w3 = result_w3.dropna(axis = 1)
        
        result_w1.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}冬季上午.csv".format(file), encoding = "utf8")
        result_w2.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}冬季中午.csv".format(file), encoding = "utf8")
        result_w3.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}冬季晚上.csv".format(file), encoding = "utf8")
        
        result_s1.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}夏季上午.csv".format(file), encoding = "utf8")
        result_s2.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}夏季中午.csv".format(file), encoding = "utf8")
        result_s3.to_csv(r"E:\D4SG\Data\PM2.5 airbox\各縣市原始資料\{}夏季晚上.csv".format(file), encoding = "utf8")       